# Automated segmentation of thorax CT scans using neural networks

### CS 180 MP 2
### Rianna Patricia Cruz

## Introduction 
The identification of tumors and abnormal masses in medical imaging scans is one of the most important steps in the treatment of such abnormal conditions. Such a essential step in the treatment process is often tedious and time-consuming, as it requires a medical professional to manually identify such masses igiven some medical images such as CT or MRI scans. Though such identification process might be straightforward for a medical professional, the time it takes for such scans to be manually and individually processed and analyzed by a medical professional causes delays which might be critical to a patient's treatment.

The objective of this study was to automatically segment four organs-at-risk (OAR): the heart, the aorta, the trachea, and the esophagus


In [35]:
import datasets
import numpy as np

In [ ]:
# print("[INFO] loading scan attributes...")
# ave_depth = get_ave_depth()
# memoize: 186.
ave_depth = 186
scans, masks = datasets.load_scans(4, ave_depth)
print("done loading data")

In [4]:
scans.shape
masks.shape

(4, 512, 512, 186)

In [92]:
from sklearn.model_selection import train_test_split
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("main: splitting data...")
split = train_test_split(scans, masks, test_size=0.75, random_state=42)
(X_train, X_test, y_train, y_test) = split
print("main: done split")

main: splitting data...
main: done split


In [93]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(1, 512, 512, 186)
(1, 512, 512, 186)
(3, 512, 512, 186)
(3, 512, 512, 186)
(512, 512, 186)
(512, 512, 186)
(512, 512, 186)
(512, 512, 186)


In [94]:
# transpose for the sake of splitting training and test data

X_train.shape
X_test.shape

(3, 512, 512, 186)

In [147]:
X_train = np.expand_dims(X_train, axis=4)
y_train = np.expand_dims(y_train, axis=4)
X_test = np.expand_dims(X_test, axis=4)
y_test = np.expand_dims(y_test, axis=4)

In [148]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(1, 512, 512, 186, 1, 1)
(1, 512, 512, 186, 1, 1)
(3, 512, 512, 186, 1, 1)
(3, 512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)


In [207]:
# https://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/
from keras.models import Sequential
from keras.layers import Conv3D, Dense, Flatten, MaxPooling3D, Dropout
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import Input

num_classes = 2

input_shape = (512, 512, 186, 1)

model = Sequential()
model.add(Conv3D(32, kernel_size=(5, 5, 5), strides=(1, 1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))
model.add(Conv3D(64, (5, 5, 5), 
                 activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
'''
input_shape=(512, 512, 1)
model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

'''
print(model.summary())

NameError: name 'x_train' is not defined

In [199]:
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import SGD

model.compile(loss=sparse_categorical_crossentropy,
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

In [200]:
from keras.callbacks import Callback
class AccuracyHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [204]:
'''
X_train_try = X_train.reshape(-1, 512, 512, 1)
X_test_try = X_test.reshape(-1, 512, 512, 1)
y_train_try = y_train.reshape(-1, 512, 512, 1)
y_test_try = y_test.reshape(-1, 512, 512, 1)
'''


In [205]:
print(X_train_try.shape)
print(y_train_try.shape)
print(X_test_try.shape)
print(y_test_try.shape)

print(X_train_try.shape[1:])
print(y_train_try.shape[1:])
print(X_test_try.shape[1:])
print(y_test_try.shape[1:])

(1, 512, 512, 186, 1, 1)
(1, 512, 512, 186, 1, 1)
(3, 512, 512, 186, 1, 1)
(3, 512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)
(512, 512, 186, 1, 1)


In [211]:
batch_size = 50
epochs = 20



model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=128)
'''
history = AccuracyHistory()

model.fit(X_train_try, y_train_try,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_try, y_test_try),
          callbacks=[history])
'''

ValueError: Error when checking input: expected dense_38_input to have 2 dimensions, but got array with shape (1, 512, 512, 186, 1, 1)